# 수정 사항
- 유진벗이 올려주신 코드에서 128개 유닛을 가진 LSTM을 사용한다고 작성된 코드 부분이 있는데, 코드에는 64개 유닛이 사용되어 있어서 수정해야 할 것 같습니다. 실험에서는 기본 LSTM 모델의 유닛을 64개로 설정하고 유닛 수를 32개, 128개로 바꾸어가며 실험을 수행했습니다.
- 불용어 사전에 '다', '은', '는' 같은 단어가 빠져 있어서, 해당 단어들을 불용어 사전 파일에 추가하여 실험을 수행하였습니다. 깃허브에 코드를 올려두었으니 참고해주시면 좋을 것 같습니다.

In [1]:
from __future__ import annotations
from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from keras_preprocessing.sequence import pad_sequences

In [2]:
# 토크나이저 생성
tokenizer = Okt()

# 데이터셋 불러오기
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")
title_name=pd.read_csv("../data/music_info.csv")

In [3]:
train_data

,Unnamed: 0,lyric,label
0,0,생각이 많은 건 말이야,0
1,1,당연히 해야 할 일이야,1
2,2,나에겐 우리가 지금 1순위야,1
3,3,안전한 유리병을 핑계로,-1
4,4,바람을 가둬 둔 것 같지만,1
...,...,...,...
2318,3187,널 위해 살아갈게,1
2319,3188,나약한 마음 따윈 모두 버릴게,1
2320,3189,우리의 사랑을 위해,1
2321,3190,너의 손을 잡고 놓지 않을게,-1


In [3]:
# 불용어 사전 로드
file_path="../words/stopwords-ko.txt"

with open(file_path,encoding='utf-8') as f:
  lines = f.readlines()
lines = [line.rstrip('\n') for line in lines]

stopwords = lines

# 훈련 데이터 전처리
okt = Okt()
X_train = []
for lyrics in train_data['lyric']:
  temp_X = []
  temp_X = okt.morphs(lyrics, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_train.append(temp_X)

# 테스트 데이터 전처리
X_test = []
for lyrics in test_data['lyrics']:
  temp_X = []
  temp_X = okt.morphs(lyrics, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

# 토큰화된 데이터의 일부 출력
print(X_train[100:103])
print(X_test[100:103])

[['손자', '돌려주다', '꼭'], ['난', '내', '놈', '씩'], ['이루다', '점점', '끝없이']]
[['짓', '없이', '그냥', '말', '만', '많다'], ['제발', '그만', '나오다', '"'], ['"', '노래', '못', '싱잉']]


In [14]:
X_train

[['생각', '많다', '건', '말', '이야'],
 ['당연하다', '일이'],
 ['에겐', '지금', '1', '순위'],
 ['안전하다', '유리', '병', '핑계'],
 ['바람', '가두다', '두다'],
 ['기억나다', '?', '그날'],
 ['잡다', '손', '엔', '말', '이야'],
 ['설레임', '보다', '커다랗다', '믿음', '담기다'],
 ['난', '함박', '웃음', '지다'],
 ['울음', '날'],
 ['소중하다', '건', '언제나', '두려움', '이니까'],
 ['문', '열다', '들리다', '목소리'],
 ['인하다', '변하다', '따뜻하다', '공기'],
 ['여전하다', '없다', '안녕하다'],
 [',', '사라지다', '별', '자리'],
 ['아스라이', '하얗다', '빛'],
 ['한', '꺼내다', '볼', '거야'],
 ['아낌없이', '반짝이다'],
 ['조금씩', '옅다', '가다'],
 ['내', '맘', '살', '숨', '쉬다', '테', '니'],
 ['여기다', ',', '서로', '끝', '아니다'],
 ['새롭다', '길', '모퉁이'],
 ['익숙하다', '진심', '속', '이지', '말'],
 ['추억', '떠오르다'],
 ['많이', '많이', '그립다', '거야'],
 ['고맙다', '이제'],
 ['사건', '지평선', '너머'],
 ['솔직하다', '두렵다', '기도'],
 ['노력', '정답', '아니다'],
 ['마지막', '선물', '산뜻하다', '안녕'],
 ['가렵지', '않다'],
 ['내', '뒤', '말', '많다'],
 ['말', '더', '필요하다'],
 ['무시', '마', '내', '걸어오다', '커리어'],
 ['더', '높이', '가줄'],
 ['내', '바르다', '세계', '젤', '위'],
 ['떨어지다', '돼다'],
 ['멋대로', '정', '나다', '애', '대해'],
 ['뜨겁다', '관심', '환영'],
 ['내',

In [4]:
# 토큰화 설정 및 토큰화 진행
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test) 
print(X_train[100:103])
print(X_test[100:103])

[[1104, 1105, 180], [3, 1, 323, 261], [406, 267, 712]]
[[854, 71, 118, 6, 10, 76], [1886, 874, 120, 965], [965, 100, 8]]


In [5]:
# 라벨 데이터 준비
y_train = []
y_test = []

# 원-핫 인코딩
for i in range(len(train_data['label'])):
  if train_data['label'].iloc[i] == 1:
    y_train.append([0, 0, 1])
  elif train_data['label'].iloc[i] == 0:
    y_train.append([0, 1, 0])
  elif train_data['label'].iloc[i] == -1:
    y_train.append([1, 0, 0])

for i in range(len(test_data['label'])):
  if test_data['label'].iloc[i] == 1:
    y_test.append([0, 0, 1])
  elif test_data['label'].iloc[i] == 0:
    y_test.append([0, 1, 0])
  elif test_data['label'].iloc[i] == -1:
    y_test.append([1, 0, 0])

y_train = np.array(y_train)
y_test = np.array(y_test)

print(y_train[100:103])
print(y_test[100:103])

[[0 0 1]
 [0 0 1]
 [1 0 0]]
[[0 1 0]
 [0 0 1]
 [0 0 1]]


## 기존 모델
- 원핫인코딩 이용
- LSTM (64개 unit, hidden-layer 1개)
- optimizer: adam
- loss: categorical_crossentropy
- 조기종료 patience=5

In [6]:
# 데이터 길이 맞추기
max_len = 12 # 전체 데이터의 길이를 12로 맞춤
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [7]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 12s 110ms/step - loss: 1.0461 - accuracy: 0.4688 - val_loss: 1.0074 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 6s 105ms/step - loss: 0.9797 - accuracy: 0.5005 - val_loss: 0.9222 - val_accuracy: 0.5892
Epoch 3/20
59/59 [==============================] - 7s 116ms/step - loss: 0.7508 - accuracy: 0.7201 - val_loss: 0.8289 - val_accuracy: 0.6043
Epoch 4/20
59/59 [==============================] - 6s 108ms/step - loss: 0.4581 - accuracy: 0.8558 - val_loss: 0.8536 - val_accuracy: 0.6129
Epoch 5/20
59/59 [==============================] - 6s 100ms/step - loss: 0.2784 - accuracy: 0.9187 - val_loss: 0.9433 - val_accuracy: 0.6452
Epoch 6/20
59/59 [==============================] - 6s 98ms/step - loss: 0.1842 - accuracy: 0.9446 - val_loss: 1.2294 - val_accuracy: 0.6043
Epoch 7/20
59/59 [==============================] - 6s 94ms/step - loss: 0.1573 - accuracy: 0.9499 - val_loss: 1.0916 - val_accuracy: 0.6065
Epoch 8

In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 7ms/step - loss: 0.5171 - accuracy: 0.8497

 테스트 정확도 : 84.97%


# 1. optimizer 변경
- 기존 모델: Adam 사용
- 실험: Nadam, SGD, RmsProp

## 1.1) Nadam

In [8]:
# 모델 컴파일
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 11s 83ms/step - loss: 1.0474 - accuracy: 0.4709 - val_loss: 1.0094 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 4s 67ms/step - loss: 1.0019 - accuracy: 0.5005 - val_loss: 0.9480 - val_accuracy: 0.5527
Epoch 3/20
59/59 [==============================] - 4s 72ms/step - loss: 0.8675 - accuracy: 0.6195 - val_loss: 0.8790 - val_accuracy: 0.6151
Epoch 4/20
59/59 [==============================] - 4s 74ms/step - loss: 0.6152 - accuracy: 0.7632 - val_loss: 0.8906 - val_accuracy: 0.5699
Epoch 5/20
59/59 [==============================] - 4s 72ms/step - loss: 0.4113 - accuracy: 0.8633 - val_loss: 0.8373 - val_accuracy: 0.6301
Epoch 6/20
59/59 [==============================] - 3s 58ms/step - loss: 0.2768 - accuracy: 0.9020 - val_loss: 1.1260 - val_accuracy: 0.5806
Epoch 7/20
59/59 [==============================] - 3s 58ms/step - loss: 0.1919 - accuracy: 0.9370 - val_loss: 1.0643 - val_accuracy: 0.6237
Epoch 8/20
5

In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 0.5629 - accuracy: 0.8493

 테스트 정확도 : 84.93%


## 1.2) SGD

In [8]:
# 모델 컴파일
from keras.optimizers import gradient_descent_v2

sgd = gradient_descent_v2.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
59/59 [==============================] - 6s 32ms/step - loss: 1.0756 - accuracy: 0.4580 - val_loss: 1.0498 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 1s 23ms/step - loss: 1.0519 - accuracy: 0.4736 - val_loss: 1.0287 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 1s 19ms/step - loss: 1.0393 - accuracy: 0.4736 - val_loss: 1.0198 - val_accuracy: 0.5419
Epoch 4/20
59/59 [==============================] - 1s 19ms/step - loss: 1.0406 - accuracy: 0.4736 - val_loss: 1.0170 - val_accuracy: 0.5419
Epoch 5/20
59/59 [==============================] - 1s 20ms/step - loss: 1.0361 - accuracy: 0.4736 - val_loss: 1.0156 - val_accuracy: 0.5419
Epoch 6/20
59/59 [==============================] - 1s 21ms/step - loss: 1.0351 - accuracy: 0.4736 - val_loss: 1.0155 - val_accuracy: 0.5419
Epoch 7/20
59/59 [==============================] - 1s 20ms/step - loss: 1.0346 - accuracy: 0.4736 - val_loss: 1.0139 - val_accuracy: 0.5419
Epoch 8/20
59

In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 1.0328 - accuracy: 0.4954

 테스트 정확도 : 49.54%


## 1.3) RmsProp

In [9]:
# 모델 컴파일
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 8s 58ms/step - loss: 1.0390 - accuracy: 0.4693 - val_loss: 0.9979 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 3s 46ms/step - loss: 0.9884 - accuracy: 0.4876 - val_loss: 1.0730 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 3s 48ms/step - loss: 0.8645 - accuracy: 0.6060 - val_loss: 0.8757 - val_accuracy: 0.6022
Epoch 4/20
59/59 [==============================] - 3s 50ms/step - loss: 0.6834 - accuracy: 0.7201 - val_loss: 0.8329 - val_accuracy: 0.6108
Epoch 5/20
59/59 [==============================] - 3s 48ms/step - loss: 0.5399 - accuracy: 0.8084 - val_loss: 0.8232 - val_accuracy: 0.6495
Epoch 6/20
59/59 [==============================] - 3s 52ms/step - loss: 0.4412 - accuracy: 0.8563 - val_loss: 0.8676 - val_accuracy: 0.6710
Epoch 7/20
59/59 [==============================] - 3s 46ms/step - loss: 0.3391 - accuracy: 0.8778 - val_loss: 1.1150 - val_accuracy: 0.6559
Epoch 8/20
59

In [10]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 7ms/step - loss: 0.6059 - accuracy: 0.8207

 테스트 정확도 : 82.07%


Adam과 Nadam, SGD, RmsProp을 비교한 결과 Adam에서의 정확도가 84.97%로 가장 높은 것을 확인할 수 있었다. 따라서, 최적의 optimizer는 Adam이라고 할 수 있을 것이다.

# 2. loss function
- 기존 모델: categorical crossentropy
- 실험: MSE, MAE, Huber loss

## 2.1) MSE

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 14s 107ms/step - loss: 0.2119 - accuracy: 0.4731 - val_loss: 0.1980 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 5s 88ms/step - loss: 0.1955 - accuracy: 0.5312 - val_loss: 0.1809 - val_accuracy: 0.6086
Epoch 3/20
59/59 [==============================] - 5s 92ms/step - loss: 0.1436 - accuracy: 0.6895 - val_loss: 0.1577 - val_accuracy: 0.6387
Epoch 4/20
59/59 [==============================] - 5s 91ms/step - loss: 0.0875 - accuracy: 0.8369 - val_loss: 0.1659 - val_accuracy: 0.6366
Epoch 5/20
59/59 [==============================] - 5s 92ms/step - loss: 0.0507 - accuracy: 0.9133 - val_loss: 0.1643 - val_accuracy: 0.6473
Epoch 6/20
59/59 [==============================] - 5s 92ms/step - loss: 0.0367 - accuracy: 0.9376 - val_loss: 0.1790 - val_accuracy: 0.6452
Epoch 7/20
59/59 [==============================] - 6s 94ms/step - loss: 0.0312 - accuracy: 0.9462 - val_loss: 0.1966 - val_accuracy: 0.6387
Epoch 8/20


In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 0.0784 - accuracy: 0.8470

 테스트 정확도 : 84.70%


## 2.2) MAE

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 11s 105ms/step - loss: 0.3882 - accuracy: 0.4661 - val_loss: 0.3055 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 5s 92ms/step - loss: 0.3509 - accuracy: 0.4736 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 5s 90ms/step - loss: 0.3508 - accuracy: 0.4736 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 4/20
59/59 [==============================] - 5s 91ms/step - loss: 0.3508 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 5/20
59/59 [==============================] - 5s 92ms/step - loss: 0.3508 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 6/20
59/59 [==============================] - 6s 96ms/step - loss: 0.3508 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 7/20
59/59 [==============================] - 5s 93ms/step - loss: 0.3507 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 8/20


In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 6ms/step - loss: 0.3362 - accuracy: 0.4958

 테스트 정확도 : 49.58%


## 2.3) Huber loss

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='huber_loss', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 11s 106ms/step - loss: 0.1053 - accuracy: 0.4699 - val_loss: 0.1010 - val_accuracy: 0.5441
Epoch 2/20
59/59 [==============================] - 5s 89ms/step - loss: 0.0982 - accuracy: 0.5199 - val_loss: 0.0924 - val_accuracy: 0.5892
Epoch 3/20
59/59 [==============================] - 5s 91ms/step - loss: 0.0750 - accuracy: 0.6873 - val_loss: 0.0851 - val_accuracy: 0.6108
Epoch 4/20
59/59 [==============================] - 5s 91ms/step - loss: 0.0465 - accuracy: 0.8294 - val_loss: 0.0857 - val_accuracy: 0.5849
Epoch 5/20
59/59 [==============================] - 5s 91ms/step - loss: 0.0282 - accuracy: 0.9064 - val_loss: 0.0830 - val_accuracy: 0.6559
Epoch 6/20
59/59 [==============================] - 6s 99ms/step - loss: 0.0177 - accuracy: 0.9354 - val_loss: 0.0873 - val_accuracy: 0.6194
Epoch 7/20
59/59 [==============================] - 5s 90ms/step - loss: 0.0144 - accuracy: 0.9446 - val_loss: 0.0957 - val_accuracy: 0.6215
Epoch 8/20


In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 6ms/step - loss: 0.0414 - accuracy: 0.8389

 테스트 정확도 : 83.89%


Categorical cross-entropy를 사용할 때의 정확도가 다른 실험에서의 정확도보다 높은 84.97%를 보이고 있다. 따라서 최적의 loss function은 categorical cross-entropy라고 볼 수 있다.

# 3. LSTM architecture 변경

## 3.1) hidden layer 수 변경

### 3.1.1) hidden layer = 2

In [10]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [11]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 18s 118ms/step - loss: 1.0490 - accuracy: 0.4682 - val_loss: 1.0236 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - ETA: 0s - loss: 1.0201 - accuracy: 0.48 - 6s 104ms/step - loss: 1.0201 - accuracy: 0.4876 - val_loss: 0.9858 - val_accuracy: 0.5484
Epoch 3/20
59/59 [==============================] - 6s 105ms/step - loss: 0.8769 - accuracy: 0.6136 - val_loss: 0.8435 - val_accuracy: 0.6194
Epoch 4/20
59/59 [==============================] - ETA: 0s - loss: 0.6038 - accuracy: 0.76 - 7s 112ms/step - loss: 0.6038 - accuracy: 0.7610 - val_loss: 0.8327 - val_accuracy: 0.6301
Epoch 5/20
59/59 [==============================] - ETA: 0s - loss: 0.3746 - accuracy: 0.86 - 7s 112ms/step - loss: 0.3746 - accuracy: 0.8644 - val_loss: 0.9099 - val_accuracy: 0.6280
Epoch 6/20
59/59 [==============================] - 7s 116ms/step - loss: 0.2526 - accuracy: 0.9101 - val_loss: 1.0153 - val_accuracy: 0.6409
Epoch 7/20
59/59 [===

In [12]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 9ms/step - loss: 0.5522 - accuracy: 0.8504

 테스트 정확도 : 85.04%


### 3.1.2) hidden layer = 3

In [13]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [14]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 24s 144ms/step - loss: 1.0450 - accuracy: 0.4666 - val_loss: 1.0073 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 7s 119ms/step - loss: 1.0315 - accuracy: 0.4736 - val_loss: 1.0168 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 7s 125ms/step - loss: 0.9642 - accuracy: 0.5301 - val_loss: 0.9204 - val_accuracy: 0.5957
Epoch 4/20
59/59 [==============================] - 8s 132ms/step - loss: 0.7478 - accuracy: 0.6593 - val_loss: 0.8451 - val_accuracy: 0.6323
Epoch 5/20
59/59 [==============================] - 8s 128ms/step - loss: 0.5673 - accuracy: 0.7379 - val_loss: 0.8916 - val_accuracy: 0.6280
Epoch 6/20
59/59 [==============================] - 8s 129ms/step - loss: 0.4305 - accuracy: 0.8337 - val_loss: 0.9789 - val_accuracy: 0.6172
Epoch 7/20
59/59 [==============================] - 8s 130ms/step - loss: 0.3561 - accuracy: 0.8671 - val_loss: 1.0439 - val_accuracy: 0.6237
Epoch

In [15]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 11ms/step - loss: 0.5565 - accuracy: 0.8431

 테스트 정확도 : 84.31%


layer 수가 2개일 때 정확도가 85.04%로 증가하였으므로 최적의 layer 수는 2개이다.

## 3.2) LSTM unit 수 변경

### 3.2.1) unit=32

In [16]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(32, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(32, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [17]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 17s 135ms/step - loss: 1.0490 - accuracy: 0.4688 - val_loss: 1.0082 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 6s 98ms/step - loss: 1.0284 - accuracy: 0.4693 - val_loss: 0.9992 - val_accuracy: 0.5441
Epoch 3/20
59/59 [==============================] - 6s 100ms/step - loss: 0.9649 - accuracy: 0.5307 - val_loss: 0.9224 - val_accuracy: 0.5871
Epoch 4/20
59/59 [==============================] - 6s 101ms/step - loss: 0.8059 - accuracy: 0.6410 - val_loss: 0.9003 - val_accuracy: 0.5570
Epoch 5/20
59/59 [==============================] - 6s 106ms/step - loss: 0.6509 - accuracy: 0.7137 - val_loss: 0.9055 - val_accuracy: 0.6237
Epoch 6/20
59/59 [==============================] - 6s 109ms/step - loss: 0.5388 - accuracy: 0.7567 - val_loss: 0.9212 - val_accuracy: 0.5935
Epoch 7/20
59/59 [==============================] - 6s 106ms/step - loss: 0.4282 - accuracy: 0.8450 - val_loss: 0.9991 - val_accuracy: 0.6215
Epoch 

In [18]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 8ms/step - loss: 0.5420 - accuracy: 0.8304

 테스트 정확도 : 83.04%


### 3.2.2) unit=128

In [19]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(128, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [20]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 18s 152ms/step - loss: 1.0483 - accuracy: 0.4682 - val_loss: 1.0093 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 8s 133ms/step - loss: 1.0222 - accuracy: 0.4806 - val_loss: 0.9756 - val_accuracy: 0.5548
Epoch 3/20
59/59 [==============================] - 8s 140ms/step - loss: 0.8667 - accuracy: 0.6125 - val_loss: 0.8758 - val_accuracy: 0.5828
Epoch 4/20
59/59 [==============================] - 9s 147ms/step - loss: 0.5907 - accuracy: 0.7589 - val_loss: 0.8180 - val_accuracy: 0.6387
Epoch 5/20
59/59 [==============================] - 9s 145ms/step - loss: 0.3656 - accuracy: 0.8654 - val_loss: 0.9487 - val_accuracy: 0.6194
Epoch 6/20
59/59 [==============================] - 8s 143ms/step - loss: 0.2529 - accuracy: 0.9096 - val_loss: 1.0429 - val_accuracy: 0.6366
Epoch 7/20
59/59 [==============================] - 8s 144ms/step - loss: 0.2006 - accuracy: 0.9241 - val_loss: 1.1838 - val_accuracy: 0.6129
Epoch

In [21]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 2s 24ms/step - loss: 0.5500 - accuracy: 0.8500

 테스트 정확도 : 85.00%


unit을 32, 64, 128로 바꾸어가며 실험한 결과 unit=64일 때 최적의 결과가 나타난다는 것을 관찰할 수 있었다.

# 4. 모델 변경

## 4.1) SVM

In [22]:
from sklearn import datasets, svm, pipeline

In [23]:
linear_svm = svm.LinearSVC()
poly_svm = svm.SVC(kernel = 'poly')
rbf_svm = svm.SVC(kernel = 'rbf')
sigmoid_svm = svm.SVC(kernel = 'sigmoid')

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
# Linear Kernel
linear_svm.fit(X_train, train_data['label'])
y_pred = linear_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# Polynomial Kernel (default:3)
poly_svm.fit(X_train, train_data['label'])
y_pred = poly_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# RBF Kernel
rbf_svm.fit(X_train, train_data['label'])
y_pred = rbf_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# Sigmoid Kernel
sigmoid_svm.fit(X_train, train_data['label'])
y_pred = sigmoid_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

C:\Users\이하은\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.3497
Accuracy: 0.5146
Accuracy: 0.5262
Accuracy: 0.3909


SVM 모델을 이용한 경우 정확도는 LSTM을 이용했을 때보다 떨어지는 것을 확인할 수 있었다.

## 4.2) Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
lr_tf = LogisticRegression(max_iter=3000) 
lr_tf.fit(X_train, train_data['label']) # 학습
y_pred = lr_tf.predict(X_test)
 
from sklearn.metrics import accuracy_score
print('Misclassified samples: {} out of {}'.format((y_pred != test_data['label']).sum(),len(test_data['label'])))
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

Misclassified samples: 1292 out of 2594
Accuracy: 0.5019


Logistic Regression을 이용한 경우에도 역시 정확도가 0.5019로 LSTM을 이용했을 때보다 확연히 떨어지는 것을 확인할 수 있다.